In [19]:
first_exec=True
first_download=True

In [1]:
#import librairies 

import tensorflow as tf
import pandas as pd 
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import progressbar
import time
from time import process_time
import sys
import os
import random
import gc
    
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn_som.som import SOM
import networkx as nx
from networkx.algorithms import approximation

pd.set_option('display.max_columns', 100)

from GEM.gem.utils      import graph_util, plot_util
from GEM.gem.evaluation import visualize_embedding as viz
from GEM.gem.evaluation import evaluate_graph_reconstruction as gr
from GEM.gem.embedding.gf       import GraphFactorization
#from GEM.gem.embedding.sdne     import SDNE
#from argparse import ArgumentParser
#from GraphEmbedding.ge import DeepWalk
#from GraphEmbedding.ge import SDNE
from karateclub.graph_embedding import Graph2Vec
from karateclub.node_embedding.neighbourhood import HOPE
from karateclub.node_embedding.neighbourhood import DeepWalk




num_frame=200#arbitraire , a tester plus serieusement

print("import terminé a :",time.strftime("%H:%M:%S", time.localtime()))

import terminé a : 12:30:52


In [16]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def deprecated_nn():
    # manual f1 score 




    #metrics=["accuracy",f1]
    start = time.time()

    model = Sequential()
    model.add(Dense(32, input_shape=(len(xtrain_transformed_complique.columns),),
                    activation='relu')),
    model.add(BatchNormalization()),
    model.add(Dense(16, activation='relu')),
    model.add(Dense(8, activation='relu')),
    model.add(Dropout(0.2)),
    model.add(Dense(4, activation='relu')),
    model.add(Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.001) #optimizer
    los=tf.keras.losses.BinaryCrossentropy()

    model.compile(optimizer=opt, loss="binary_crossentropy", 
                  metrics=["accuracy",f1])


    history= model.fit(xtrain_transformed_complique, ytrain_transformed_complique
                       ,epochs = 10, batch_size=128, verbose = 0)
                       #,class_weight=weight)#didnt work
    history_dictict = history.history
    #sigmoid return values from 0 to 1 , not exactly 0 and 1
    y_train_pred=(model.predict(xtrain_transformed_complique)>0.5).astype("int32")
    y_test_pred =(model.predict(xtest_transformed_complique) >0.5).astype("int32")

    print(classification_report(ytrain_transformed_complique,y_train_pred))
    print(classification_report(ytest_transformed_complique,y_test_pred))

    cl_nn=classification_report(ytest_transformed_complique,y_test_pred
                                ,output_dict=True)
    return cl_nn

print("declaration de fonction terminé a :",time.strftime("%H:%M:%S", time.localtime()))

declaration de fonction terminé a : 12:37:40


In [17]:
if(first_download):    
    #import le data_set complet --- import data_set = 28.00195622444153 seconds ---
    start = time.time()

    data_file= os.path.abspath('data')
    full_path=data_file+'\\'+'fraudTrain.csv'   # unmodified train set
    train_df=pd.read_csv(full_path)
    full_path=data_file+'\\'+'fraudTest.csv'
    test_df=pd.read_csv(full_path)



    full_path=data_file+'\\'+'X_train_1_2_svm.csv'
    xtrain_transformed_complique=pd.read_csv(full_path)
    ytrain_transformed_complique=train_df['is_fraud'].iloc[:int(len(train_df)*0.8)]

    full_path=data_file+'\\'+'X_val_1_2_svm.csv'
    xval_transformed_complique=pd.read_csv(full_path)
    yval_transformed_complique=train_df['is_fraud'].iloc[len(ytrain_transformed_complique):]


    full_path=data_file+'\\'+'X_test_1_2_svm.csv'
    xtest_transformed_complique=pd.read_csv(full_path)
    ytest_transformed_complique=test_df['is_fraud']


    train_df=train_df.drop(columns=['Unnamed: 0'])
    test_df=test_df.drop(columns=['Unnamed: 0'])
    xtrain_transformed_complique=xtrain_transformed_complique.drop(columns=['Unnamed: 0'])
    xval_transformed_complique=xval_transformed_complique.drop(columns=['Unnamed: 0'])
    xtest_transformed_complique=xtest_transformed_complique.drop(columns=['Unnamed: 0'])


    cols = xtrain_transformed_complique.columns.tolist()
    print(cols)
    first_download=False
    
print("--- import data_set = %s seconds ---" % (time.time() - start));start = time.time()

--- import data_set = 323.01495718955994 seconds ---


In [18]:
cl=deprecated_nn()

print("choosen model precision: %4.f" %cl["1"]["precision"]," f1-score: %.4f" %cl["1"]['f1-score'])

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.96      0.80      0.87      5968

    accuracy                           1.00   1037340
   macro avg       0.98      0.90      0.94   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.95      0.74      0.83      2145

    accuracy                           1.00    555719
   macro avg       0.98      0.87      0.91    555719
weighted avg       1.00      1.00      1.00    555719

choosen model precision:    1  f1-score:    1


In [27]:
print("choosen model precision: %.4f" %cl["1"]["precision"]," f1-score: %.4f" %cl["1"]['f1-score'])

choosen model precision: 0.9524  f1-score: 0.8304
